In [1]:
import pandas as pd
from openpyxl import load_workbook
from tqdm.notebook import tqdm

In [2]:
def get_sheet_names(filename):
    wb = load_workbook(filename + ".xlsx")
    return wb.sheetnames

In [3]:
def load_sheet(filename, sheetname):
    
    wb = load_workbook(filename=filename+'.xlsx')
    sheet = wb[sheetname]
    data = []

    headers = [cell.value for cell in sheet[1]]

    for row in sheet.iter_rows(min_row=2):
        row_dict = {}
        for key, cell in zip(headers, row):
            row_dict[key] = cell.value

        data.append(row_dict)
    return data

In [4]:
filename = "NPS_04"

In [5]:
# email e empresa
sheets = get_sheet_names(filename)
for sheet in sheets:
    form = ['APP TERCEIRAS','WEB TERCEIRAS (RODOVIAS)']
    print("running", sheet)
    
    if sheet not in form:
        print("skipping", sheet)
        continue
    
    data = load_sheet(filename, sheet)
    for a in tqdm(data):
        try:
            user = User.objects.filter(uuid=a['user.userId'])
            empresa = UserInFirm.objects.filter(user=user.first()).values('firm__subcompany__name')
            if empresa.exists() and a['rating'] > 8:
                empresa=empresa.first()
                user=user.first()
                a['email']=user.email
                a['Empresa Terceira'] = empresa['firm__subcompany__name']
        except Exception as e:
            print(e)
    if "APP TERCEIRAS" in sheet:
        df_app = pd.DataFrame(data=data)
    elif "WEB TERCEIRAS (RODOVIAS)" in sheet:
        df_web = pd.DataFrame(data=data)

running APP TERCEIRAS


  0%|          | 0/14 [00:00<?, ?it/s]

running WEB TERCEIRAS (RODOVIAS)


  0%|          | 0/13 [00:00<?, ?it/s]

running Terceiros Dados
skipping Terceiros Dados
running NPS
skipping NPS
running APP GERAL (ANTIGA)
skipping APP GERAL (ANTIGA)
running WEB GERAL (RODOVIAS) (ANTIGA)
skipping WEB GERAL (RODOVIAS) (ANTIGA)
running WEB GERAL (ENERGIA)
skipping WEB GERAL (ENERGIA)


In [6]:
df_origin = pd.read_excel(filename+'.xlsx',sheet_name='Terceiros Dados')

In [7]:
df_concatenada = pd.concat([df_origin,df_app,df_web])

In [17]:
df_concatenada = df_concatenada[~df_concatenada['id'].isnull()]
df_concatenada

,id,data created,traits.unidade,rating,feedback,Você autoriza que façamos contato para entender mais sobre o seu feedback?,Deixe um feedback para nós! Gostamos muito de ouvir nossos clientes!,method,device_type,device_os,...,Equipe,E-mail,Empresa,campaign.id,campaign.name,email,Empresa Terceira,Unnamed: 31,created,"Em sua opinião, o que poderíamos fazer para melhorar? (resposta não obrigatória)"
0,624c41b3afe2996d2a05820b,2022-04-05T13:18:43.775Z,Eixo SP,10,-,-,-,In-app,desktop,Windows,...,-,-,-,624c3f2192a80e10ce74bc8b,NPS web,atilio.coradine@eixosp.com.br,MRT - EIXO SP,-,-,-
1,624c5802afe2996d2a059a28,2022-04-05T14:53:54.046Z,Entrevias,10,-,-,-,In-app,desktop,Windows,...,-,-,-,624c3f2192a80e10ce74bc8b,NPS web,alexandre.bassi@entrevias.com.br,CENTRAL SINALIZAÇÕES,-,-,-
2,624c70f7afe2996d2a05b285,2022-04-05T16:40:23.991Z,CART,10,-,-,-,In-app,desktop,Windows,...,-,-,-,624c3f2192a80e10ce74bc8b,NPS web,Vinicius.biel@cartsp.com.br,CART - Sinalização,-,-,-
3,624c7414afe2996d2a05b553,2022-04-05T16:53:40.803Z,Arteris Via Paulista,10,-,-,-,In-app,desktop,Windows,...,-,-,-,624c3f2192a80e10ce74bc8b,NPS web,silas.mello@arteris.com.br,-,-,-,-
4,624c75eaafe2996d2a05b70b,2022-04-05T17:01:30.966Z,Arteris Fernão Dias,10,-,-,-,In-app,desktop,Windows,...,-,-,-,624c3f2192a80e10ce74bc8b,NPS web,lucasvaladao.gomes@arteris.com.br,Nuno Lev,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13,660300e428d7142c73e85906,NaN,Arteris Via Paulista,10,None,NaN,NaN,Mobile,phone,Android,...,NaN,NaN,NaN,61faf98a452b125091d9e4be,NPS Survey app,marcos.santos@arteris.com.br,BC2,NaN,2024-03-26T17:07:48.341Z,NaN
0,66280ce1e16e775bb12758e9,NaN,Nova Rota do Oeste - Obras,2.0,NaN,NaN,NaN,In-app,desktop,Windows,...,NaN,NaN,NaN,624c3f2192a80e10ce74bc8b,NPS web,NaN,NaN,NaN,2024-04-23T19:32:49.336Z,None
1,6621546b795eefede1548bc3,NaN,CAJ,10.0,NaN,NaN,NaN,In-app,desktop,Windows,...,NaN,NaN,NaN,624c3f2192a80e10ce74bc8b,NPS web,jose.ribamar@ebs.eng.br,Consórcio Saneando Morro do Meio,NaN,2024-04-18T17:12:11.457Z,None
2,66214084795eefede1548482,NaN,Rota do Oeste,9.0,NaN,NaN,NaN,In-app,desktop,Windows,...,NaN,NaN,NaN,624c3f2192a80e10ce74bc8b,NPS web,belizario.engenharia@pedrisa.com.br,Pedrisa,NaN,2024-04-18T15:47:16.211Z,None


In [10]:
with pd.ExcelWriter(filename+'.xlsx', mode='a',if_sheet_exists='replace') as writer:
    df_concatenada.to_excel(writer,sheet_name='Terceiros Dados',na_rep="-", index=False)